<a href="https://colab.research.google.com/github/bmj0428/CSE6521/blob/main/TextFooler_241111.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!git clone https://github.com/jind11/TextFooler.git

%cd TextFooler


In [ ]:
!pip install nltk boto3 pattern datasets


In [1]:
%cd TextFooler

/content/TextFooler


In [ ]:
%cd ESIM
!python setup.py install
%cd ..


In [3]:
!pip install transformers==4.21.0
!pip install --upgrade tensorflow tensorflow_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 66.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.21.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification

# PyTorch 기반의 BERT 토크나이저와 모델 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# 모델과 토크나이저를 로컬 디렉토리에 저장 (safetensors 사용하지 않기)
model.save_pretrained('/content/models/bert_model', use_safetensors=False)
tokenizer.save_pretrained('/content/models/bert_model')


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:367: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

('/content/models/bert_model/tokenizer_config.json',
 '/content/models/bert_model/special_tokens_map.json',
 '/content/models/bert_model/vocab.txt',
 '/content/models/bert_model/added_tokens.json')

In [6]:
!mv /content/models/bert_model/config.json /content/models/bert_model/bert_config.json


In [7]:
!ls /content/models/bert_model/

bert_config.json  pytorch_model.bin  special_tokens_map.json  tokenizer_config.json  vocab.txt


In [8]:
from datasets import load_dataset
import pandas as pd

# Hugging Face의 'yelp_polarity' 데이터셋을 로드합니다.
dataset = load_dataset('yelp_polarity', split='train')

# 데이터셋을 판다스 데이터프레임으로 변환
df = pd.DataFrame(dataset)

# 데이터프레임의 컬럼 확인
print(df.columns)  # ['text', 'label']

# 레이블과 텍스트 컬럼만 선택, 레이블을 먼저
df = df[['label', 'text']]

# 데이터프레임에서 레이블이 정수로 되어 있는지 확인 (레이블 컬럼이 정수형이어야 함)
df['label'] = df['label'].astype(int)

# 공백으로 구분된 텍스트 파일로 저장 (레이블 먼저, 텍스트 나중)
with open('/content/yelp_reviews_no_header.txt', 'w', encoding='utf8') as f:
    for _, row in df.iterrows():
        label = row['label']
        # 텍스트 내의 개행 문자를 제거하고 공백으로 대체
        text = row['text'].replace('\n', ' ').replace('\r', ' ').strip()
        # 레이블과 텍스트를 공백으로 구분하여 저장
        f.write(f"{label} {text}\n")

# 저장된 파일 일부 확인
with open('/content/yelp_reviews_no_header.txt', 'r', encoding='utf8') as f:
    for i, line in enumerate(f):
        if i < 5:  # 첫 5줄만 출력
            print(line)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.93k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/256M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

Index(['text', 'label'], dtype='object')
0 Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars.

1 Been going to Dr. Goldberg for over 10 years. I think I was one of his 1st patients when he started at MHMG. He's been great over the years and is really all about the big picture. It is because of him, not my now former gyn Dr. Markoff, that I found out I have fibroids. He explores all options with you and i

In [9]:
%cd /content/TextFooler

/content/TextFooler


In [10]:
!python comp_cos_sim_mat.py /content/counter-fitted-vectors.txt /content/counter-fitted-cos-sim.npy 10000

Embeddings shape: (10000, 300)
Cosine similarity matrix saved to /content/counter-fitted-cos-sim.npy


In [11]:
!ls /content/


counter-fitted-cos-sim.npy  models	 TextFooler
counter-fitted-vectors.txt  sample_data  yelp_reviews_no_header.txt


In [12]:
!python attack_classification.py \
  --dataset_path /content/yelp_reviews_no_header.txt \
  --target_model bert \
  --target_model_path /content/models/bert_model/ \
  --counter_fitting_embeddings_path /content/counter-fitted-vectors.txt \
  --counter_fitting_cos_sim_path /content/counter-fitted-cos-sim.npy \
  --USE_cache_path /content/USE_cache/

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
2024-11-11 07:27:52.840549: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-11 07:27:53.069316: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-11 07:27:53.083690: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-11 07:27:53.138248: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow b